# Trust in the EU Parliament among EU citizens
*Based on the 2019 European Election Study (EES) Voter Study data*

## Notebook 1 - Data Processing



#### Introduction
In this notebook, I will clean and reshape the European Election Study (EES) data. My aim is to have it in a form such that a tree-based classification algorithm can be applied on it.



#### 1. Import packages

Let us start by importing all Python libraries that will be used below.

In [ ]:
import pandas as pd
import numpy as np
import os 

#### 2. Import data

The next step is to load the raw data and auxiliary files which will help us later.

In [ ]:
# Open the main data file.
df = pd.read_stata('EES2019 Voter Study early release.dta', 
                   convert_categoricals = False)

df.head()

C:\Users\dawid\anaconda3\lib\site-packages\pandas\io\stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


,respid,Serial,hCountry,countrycode,region,region_NUTS1,region_NUTS2,region_NUTS3,Q1_1,Q1_10,...,EDU,lang_BE,lang_est,lang_finn,lang_LT,WGT1,WGT2,WGT3,WGT4,WGT5
0,99,100009348.0,1,1040,108,AT1,AT12,96,,,...,3,NaN,NaN,NaN,NaN,1.047887,0.454922,0.909012,1.160725,1.024638
1,7762,100007762.0,1,1040,108,AT1,AT12,96,Politische Stabilität,,...,2,NaN,NaN,NaN,NaN,1.074892,0.465540,0.915341,1.582634,1.445454
2,7801,100007801.0,1,1040,108,AT1,AT12,96,Spaltung der Gesellschaft durch Hetze und Lüge...,,...,3,NaN,NaN,NaN,NaN,1.047887,0.454922,0.909012,0.768076,0.696121
3,7904,100007904.0,1,1040,102,AT3,AT33,96,mangelnder Umweltschutz,,...,97,NaN,NaN,NaN,NaN,0.867431,0.454922,0.853581,1.582634,1.093430
4,9223,100009223.0,1,1040,109,AT1,AT13,96,,,...,97,NaN,NaN,NaN,NaN,1.065503,0.553030,0.987536,1.582634,1.524754


The warning about encoding can be ignored!

We see it due to the fact that the file contains unique, language-specific  characters for Russian, Greek etc.

In [ ]:
# Import party codes and their mapping.
party_codes = pd.read_csv('party-list-appendix.csv', encoding = "latin-1") 
party_codes.head()

# Rename one column to fix a mistake in its name.
party_codes = party_codes.rename(columns={"Coutnry_short": "country_short"})

In [ ]:
# Import the NUTS names mappping.
NUTS_codes = pd.read_csv('NUTS2.csv', encoding = "utf-8") 

#### 3. Start cleaning

We can now proceed with looking at what is in the main DataFrame and processing its contents.

In [ ]:
# Which column should be the ID column?
# We have two columns ("Serial" and "respid") which could potentially be our unique IDs. The data description is quite vague here.
respid_unique_len = len(df["respid"].unique())
Serial_unique_len = len(df["Serial"].unique())
df_dims = df.shape

print('Number of unique entries of "respid": {}'.format(respid_unique_len))
print('Number of unique entries of "Serial": {}'.format(Serial_unique_len))
print('Number of rows: {}'.format(df.shape[0]))

# The documentation of the dataset indicates that more than 26,500 reponses were recorded. 
# "Serial" would be a better candidate since each row has a unique entry in that column and the length of the unique responses matches the documentation.

Number of unique entries of "respid": 25757
Number of unique entries of "Serial": 26538
Number of rows: 26538


In [ ]:
# Drop the "respid" column.
df = df.drop(columns = ["respid"])
df.head()

,Serial,hCountry,countrycode,region,region_NUTS1,region_NUTS2,region_NUTS3,Q1_1,Q1_10,Q1_11,...,EDU,lang_BE,lang_est,lang_finn,lang_LT,WGT1,WGT2,WGT3,WGT4,WGT5
0,100009348.0,1,1040,108,AT1,AT12,96,,,,...,3,NaN,NaN,NaN,NaN,1.047887,0.454922,0.909012,1.160725,1.024638
1,100007762.0,1,1040,108,AT1,AT12,96,Politische Stabilität,,,...,2,NaN,NaN,NaN,NaN,1.074892,0.465540,0.915341,1.582634,1.445454
2,100007801.0,1,1040,108,AT1,AT12,96,Spaltung der Gesellschaft durch Hetze und Lüge...,,,...,3,NaN,NaN,NaN,NaN,1.047887,0.454922,0.909012,0.768076,0.696121
3,100007904.0,1,1040,102,AT3,AT33,96,mangelnder Umweltschutz,,,...,97,NaN,NaN,NaN,NaN,0.867431,0.454922,0.853581,1.582634,1.093430
4,100009223.0,1,1040,109,AT1,AT13,96,,,,...,97,NaN,NaN,NaN,NaN,1.065503,0.553030,0.987536,1.582634,1.524754


In [ ]:
# Analyse columns with features.
# We need to investigate what some of them mean as no clear data dictionary has been provided with the dataset.
# Below, we will try to make informed guesses as to what certain columns mean if their names have not been explained in the documentation.
print(list(df.columns))

['Serial', 'hCountry', 'countrycode', 'region', 'region_NUTS1', 'region_NUTS2', 'region_NUTS3', 'Q1_1', 'Q1_10', 'Q1_11', 'Q1_12', 'noanswerQ1_98', 'Q2', 'Q2n', 'Q2_EES', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q7n', 'Q7_ees', 'Q8', 'Q9', 'Q9n', 'Q9_ees', 'q10_1', 'q10_2', 'q10_3', 'q10_4', 'q10_5', 'q10_6', 'q10_7', 'q10_8', 'q10_9', 'q10_10', 'Q11', 'q13_1', 'q13_2', 'q13_3', 'q13_4', 'q13_5', 'q13_6', 'q13_7', 'q13_8', 'q13_9', 'Q14_1', 'Q14_2', 'Q14_3', 'Q14_4', 'Q14_5', 'Q14_6', 'Q16', 'Q18_1', 'Q18_2', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'q24_1', 'q24_2', 'q24_3', 'q24_4', 'q24_5', 'q24_6', 'q24_7', 'q24_8', 'q24_9', 'Q25', 'Q25n', 'Q25_EES', 'Q26', 'Q27_1', 'Q27_2', 'Q27_3', 'QG32_1', 'QG32_2', 'QGQ34_1', 'QG34_2', 'QG36_1', 'QG36_2', 'QG38_1', 'QG38_2', 'QG38_3', 'QG38_4', 'QG38_5', 'QG38_6', 'D1', 'D4a', 'D4a_96_other', 'D4b_1', 'noanswerD4b_2', 'D5', 'D6', 'D6a', 'D7', 'D8', 'D9', 'D10', 'D11', 'D3', 'D4_1', 'hAge', 'D2_1', 'EDU', 'lang_BE', 'lang_est', 'lang_finn', 'lang_LT', 'WGT1', 

In [ ]:
# hCountry
df.groupby(by="hCountry").count()["Serial"].values == df.groupby(by="countrycode").count()["Serial"].values

# This column contains the same information (i.e. country) as "countrycode", hence, we can drop it.
df = df.drop(columns = ["hCountry"])

In [ ]:
# Columns region, region_NUTS1, region_NUTS2, region_NUTS3 relate to regions where respondents live. 
# Different columns use different ways of splitting EU + UK countries in regions.

# We will drop column region_NUTS3 since it is in majority of cases equal to 96 which does
# not mean anything in terms of NUTS codes. We will also drop region column as the encoding is
# unknown and we already have this information in our NUTS regions.
df = df.drop(columns = ["region", "region_NUTS3"])

# We will map names to NUTS codes now.
# This way, we are going to have actual names instead of codes.
mapNUTS = dict(zip(NUTS_codes["NUTS"], NUTS_codes["name"]))
df["region_NUTS1"] = df["region_NUTS1"].map(mapNUTS, na_action = "ignore")
df["region_NUTS2"] = df["region_NUTS2"].map(mapNUTS, na_action = "ignore")

In [ ]:
# Q1_10, Q1_11, Q1_12
df.groupby(by="Q1_10").count()["Serial"]
df.groupby(by="Q1_11").count()["Serial"]
df.groupby(by="Q1_12").count()["Serial"]
# They are predominantly blank and relate to Q1.

df[(df["Q1_1"] != "") & (df["Q1_10"] != "") & (df["Q1_11"] != "")][["Q1_1", "Q1_10", "Q1_11"]].values 
# It seems that if someone provided a really long answer, it was split into few columns.

# We can drop them.
df = df.drop(columns = ["Q1_10", "Q1_11", "Q1_12"])

In [ ]:
# noanswerQ1_98

# In Q1, respondents wrote the most pressing issues for their countries.
# They could answer "I don't know" as well, instead of typing anything or leaving this field blank.
# noanswerQ1_98 probably tells us how many people selected "I don't know" instead of writing anything (or not providing any response).
df.groupby(by="noanswerQ1_98").count()["Serial"]

print('Number of respondents who were marked as answered "I do not know" and left the field blank: {}'.format(df[(df["noanswerQ1_98"] == 1) & (df["Q1_1"] == "")].shape[0]))
print('Number of respondents who were marked as answered "I do not know" and did not leave the field blank: {}'.format(df[(df["noanswerQ1_98"] == 1) & (df["Q1_1"] != "")].shape[0]))
print('Number of respondents who were not marked as answered "I do not know" and left the field blank: {}'.format(df[(df["noanswerQ1_98"] != 1) & (df["Q1_1"] == "")].shape[0]))
print('Number of respondents who were not marked as answered "I do not know" and did not leave the field blank: {}'.format(df[(df["noanswerQ1_98"] != 1) & (df["Q1_1"] != "")].shape[0]))

# The result is problematic as it shows issues with the data. We cannot have people answering and leaving the field blank at the same time!
# To solve it, we will assume that leaving field blank is equivalent to saying "I don't know" and delete column noanswerQ1_98.
df = df.drop(columns = ["noanswerQ1_98"])

Number of respondents who were marked as answered "I do not know" and left the field blank: 3449
Number of respondents who were marked as answered "I do not know" and did not leave the field blank: 0
Number of respondents who were not marked as answered "I do not know" and left the field blank: 168
Number of respondents who were not marked as answered "I do not know" and did not leave the field blank: 22921


In [ ]:
# noanswerD4b_2
# Question D4b asked: IF OTHER in D4a (In which country were you born?), when did you first come to [Country]?
# It asked respondents to write down a year.
# noanswerD4b_2 probably states which respondents did not provide such answer.
df.groupby(by="noanswerD4b_2").count()["Serial"]

print('Number of respondents who were marked as "provided an answer to D4b" but noanswerD4b_2 says they did not: {}'.format(df[(df["D4b_1"].notna()) & (df["noanswerD4b_2"] == 1)].shape[0]))
print('Number of respondents who were marked as "provided an answer to D4b" and noanswerD4b_2 says they did: {}'.format(df[(df["D4b_1"].notna()) & (df["noanswerD4b_2"] == 0)].shape[0]))
print('Number of respondents who were marked as "did not provide an answer to D4b" and noanswerD4b_2 says they did not: {}'.format(df[(df["D4b_1"].isna()) & (df["noanswerD4b_2"] == 1)].shape[0]))
print('Number of respondents who were marked as "did not provide an answer to D4b" but noanswerD4b_2 says they did: {}'.format(df[(df["D4b_1"].isna()) & (df["noanswerD4b_2"] == 0)].shape[0]))

# We are encountering similar issues here as with noanswerQ1_98, hence, I will drop the column too.
df = df.drop(columns = ["noanswerD4b_2"])

Number of respondents who were marked as "provided an answer to D4b" but noanswerD4b_2 says they did not: 85
Number of respondents who were marked as "provided an answer to D4b" and noanswerD4b_2 says they did: 1561
Number of respondents who were marked as "did not provide an answer to D4b" and noanswerD4b_2 says they did not: 0
Number of respondents who were marked as "did not provide an answer to D4b" but noanswerD4b_2 says they did: 483


In [ ]:
# All columns with "n" in their name
# They can be dropped since they just help with matching parties across the dataset and with other surveys but will
# not help us with modelling or presenting the information. We can understand their role thanks to the documentation.
df = df.drop(columns = ["Q2n", "Q7n", "Q9n", "Q25n"])

In [ ]:
# All columns starting with "lang"
df.groupby(by=["countrycode", "lang_finn"]).count() # Values of 1 or 2 in Finland only.
df.groupby(by=["countrycode", "lang_est"]).count() # Values of 1 or 2 in Estonia only.
df.groupby(by=["countrycode", "lang_BE"]).count() # Values of 1 or 2 in Belgium only.
df.groupby(by=["countrycode", "lang_LT"]).count() # Values of 1 or 2 in Latvia only.
# In those countries, respondents could select in what language they wanted to answer the questions,
# there were 2 versions of national questionnaires prepared.

# However, there were 3 versions of the survey for Luxembourg (French, German, Luxembourgish)
# and that is not captured in the dataset (i.e. in what language the response was recorded).

# Let us fill those columns with 0s where we have missing values.
# This way, we can still preserve the information about the languages of respondents of those 4 countries
# but indicate that they are not relevant for the rest of our observations.
df["lang_finn"] = df["lang_finn"].fillna(0)
df["lang_est"] = df["lang_est"].fillna(0)
df["lang_BE"] = df["lang_BE"].fillna(0)
df["lang_LT"] = df["lang_LT"].fillna(0)

In [ ]:
# WGT columns
# Sampling weights used when selecting respondents to the survey used to avoid underrespresenting citizens
# who may not use the internet frequently.
# We can drop them as they do not tell us anything about our individual observations and were computed on the information
# that the dataset already contains.
df = df.drop(columns = ["WGT1", "WGT2", "WGT3", "WGT4", "WGT5"])

In [ ]:
# Replace the entries in Q2, Q7, Q9, Q25 with actual names of parties.
# Drop Q2_EES, Q7_ees, Q9_ees, Q25_EES - they capture the same information as Q2, Q7, Q9, Q25.
party_names_Q2 = party_codes[["Q2", "English name"]].drop_duplicates().dropna()
party_names_Q7 = party_codes[["Q7", "English name"]].drop_duplicates().dropna()
party_names_Q9 = party_codes[["Q9", "English name"]].drop_duplicates().dropna()
party_names_Q25 = party_codes[["q25", "English name"]].drop_duplicates().dropna()

# Skip one party as their assigned code in party_names_Q25 does not make sense.
party_names_Q25 = party_names_Q25[party_names_Q25["English name"] != "Of the Region & Pirate Party"]

# Creating mappings.
mapQ2 = dict(zip(party_names_Q2["Q2"], party_names_Q2["English name"]))
mapQ7 = dict(zip(party_names_Q7["Q7"], party_names_Q7["English name"]))
mapQ9 = dict(zip(party_names_Q9["Q9"], party_names_Q9["English name"]))

# Converting the keys to floats to match the data type in df.
mapQ25 = dict(zip([float(value.strip()) for value in party_names_Q25["q25"]], party_names_Q25["English name"]))

# Apply mappings.
df["Q2"] = df["Q2"].map(mapQ2, na_action = "ignore")
df["Q7"] = df["Q7"].map(mapQ7, na_action = "ignore")
df["Q9"] = df["Q9"].map(mapQ9, na_action = "ignore")
df["Q25"] = df["Q25"].map(mapQ25, na_action = "ignore")

# Drop the superfluous columns.
df = df.drop(columns = ["Q2_EES", "Q7_ees", "Q9_ees", "Q25_EES"])

In [ ]:
# hAge
# Varies from 1 to 5 (ordinal variable) but unexplained by the documentation and not mentioned
# in the questionnaire! We can keep it and see if it helps us with modelling.

In [ ]:
# Drop Question 26 since the entries there do not have the same
# meaning across all rows. For example, "feeling strong sympathy" for row 1
# and "feeling strong sympathy" for row 2 apply to different national parties
# while the encoded answers are identical. 
df = df.drop(columns = ["Q26"])

In [ ]:
# Replace Question 27 which asks to identify which EU Party Group supports a given candidate
# with columns that tell us whether respondents were able to correctly do it.
# Correct answer = 1
# Wrong answer/"I don't know" = 0
# Missing value = keep the missing value
df["EUPresident_correct_1"] = [1 if response == 2 else 0 if response in [1, 3, 4, 98] else np.nan for response in df['Q27_1']]
df["EUPresident_correct_2"] = [1 if response == 1 else 0 if response in [2, 3, 4, 98] else np.nan for response in df['Q27_2']]
df["EUPresident_correct_3"] = [1 if response == 4 else 0 if response in [1, 3, 2, 98] else np.nan for response in df['Q27_3']]
df = df.drop(columns = ['Q27_1', 'Q27_2', 'Q27_3'])

In [ ]:
# Questions 10, 13, 24 in the questionnaire had the following format:
# - Each row referred to a national party
# - Each column would assign a rating to that party
# - Column ratings would be identical across countries, yet rows would indicate different parties

# I reshaped the data so that each party from each country from each question is 1 column and the
# column entries tell us the rating that a given respondent assigned to a party in that question.

In [ ]:
# Find a list of all country codes.
countries = list(set(party_codes["countrycode"]))

# Get a table showing shortened country names and their codes.
country_names_mapping = party_codes[["country_short", "countrycode"]].drop_duplicates().dropna()

# Reshape answers to Q10, Q13 and Q24.
def reshape_pivots(q, q_columns, survey_data):
        '''
        The function does the reshaping outlined in the cell above. 

        Args:
            q: Name of the question.
            q_columns: List containing all column names relating to the question.
            survey_data: DataFrame which will be processed.

        Returns:
            survey_data: New DataFrame which contains the reshaped information.
        '''

    # Define what column from party_codes DataFrame should be used for mapping party names to their codes.
    if q == "Q10":
        party_col = "Q10_PTV"

    if q == "Q13":
        party_col = "Q13_left_right "

    if q == "Q24":
        party_col = "Q24_EU"

    # Get a table with country codes, party codes and their English names.
    mapping = party_codes[party_codes[party_col].notna()][["countrycode", party_col, "English name"]]
        
    # Select the responses for a given country...
    for country in countries:

        # ... select a column that shows the answers about a party in that country (in the context of the chosen question)...
        for party in q_columns:

            # (check if the column that has been picked existed in the survey for that country)
            if party.upper() in set(mapping[mapping["countrycode"] == country][party_col].values.tolist()):

                #... and create a new column linked to its ratings by that country's citizens.

                # Get the name of the party.
                col_name = mapping[(mapping["countrycode"] == country) & (mapping[party_col] == party.upper())]["English name"].tolist()[0]

                # Get the country name.
                actual_country_name = country_names_mapping[country_names_mapping["countrycode"] == country]["country_short"].tolist()[0]

                # Fill the column with NAs and ratings.
                # If someone cannot rate it (i.e. vote for it since they are not a citizen), assign NA to that row.
                # If that party has a rating, place it inside the new column.
                survey_data[str(q)+"_"+str(col_name)+"_"+str(actual_country_name)] = np.full(survey_data.shape[0], np.nan).tolist()
                survey_data.loc[survey_data["countrycode"] == country, str(q)+"_"+str(col_name)+"_"+str(actual_country_name)] = survey_data.loc[survey_data["countrycode"] == country, party]

    # Drop the original columns that related to this question.
    survey_data = survey_data.drop(columns = q_columns)
    print("Finished processing:")
    print(q)
    return survey_data

# Define which columns apply to specific questions.
Q10_parties = ['q10_1', 'q10_2', 'q10_3', 'q10_4', 'q10_5', 'q10_6', 'q10_7', 'q10_8', 'q10_9', 'q10_10']
Q13_parties = ['q13_1', 'q13_2', 'q13_3', 'q13_4', 'q13_5', 'q13_6', 'q13_7', 'q13_8', 'q13_9']
Q24_parties = ['q24_1', 'q24_2', 'q24_3', 'q24_4', 'q24_5', 'q24_6', 'q24_7', 'q24_8', 'q24_9']
pivots_columns = [Q10_parties, Q13_parties, Q24_parties]

# Reshape our main DataFrame.
df_new_Q10 = reshape_pivots("Q10", Q10_parties, df)
df_new_Q13 = reshape_pivots("Q13", Q13_parties, df_new_Q10)
df_new_Q24 = reshape_pivots("Q24", Q24_parties, df_new_Q13)
print("Done!")
df = df_new_Q24

Finished processing:
Q10
Finished processing:
Q13
Finished processing:
Q24
Done!


In [ ]:
# Drop country codes and replace them with actual names.
mapcountries = dict(zip(country_names_mapping["countrycode"], country_names_mapping["country_short"]))
df["countrycode"] = df["countrycode"].map(mapcountries, na_action = "ignore")
df["countrycode"] = df["countrycode"].map(mapNUTS, na_action = "ignore")
df.head()

,Serial,countrycode,region_NUTS1,region_NUTS2,Q1_1,Q2,Q3,Q4,Q5,Q6,...,Q24_ Imperium Europa _MT,Q24_People's Party for Freedom and Democracy_NL,Q24_Party of Freedom_NL,Q24_Christian Democratic Appeal_NL,Q24_Democrats '66_NL,Q24_Green Left_NL,Q24_Socialist Party_NL,Q24_Labour Party_NL,Q24_Christian Union_NL,Q24_Forum for Democracy_NL
0,100009348.0,ÖSTERREICH,OSTÖSTERREICH,Niederösterreich,,NaN,2,4,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100007762.0,ÖSTERREICH,OSTÖSTERREICH,Niederösterreich,Politische Stabilität,Austrian People's Party,2,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100007801.0,ÖSTERREICH,OSTÖSTERREICH,Niederösterreich,Spaltung der Gesellschaft durch Hetze und Lüge...,NaN,2,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100007904.0,ÖSTERREICH,WESTÖSTERREICH,Tirol,mangelnder Umweltschutz,The Greens,2,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100009223.0,ÖSTERREICH,OSTÖSTERREICH,Wien,,NaN,2,2,98,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Add new columns with names/text translated into English.

# Import my API key for using Google Translate and initiate the translation client.
# The key is in a private .json file.
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\dawid\Downloads\APIKEY.json"
translate_client = translate.Client()

# Translate names of countries and regions into English.
country_original = list(set(df["countrycode"].dropna()))
Country_ENG = np.array([translate_client.translate(name)['translatedText'] for name in country_original])

In [ ]:
# Fix certain issues with names in English.
Country_ENG[np.where(Country_ENG == "BELGIQUE-BELGIUM")] = "Belgium"
Country_ENG[np.where(Country_ENG == "SUOMI FINLAND")] = "Finland"

def capital_letters(name):
    name = name.split()
    name = [sub_name.capitalize() for sub_name in name]
    name = " ".join(name)
    return name
  
Country_ENG = [capital_letters(name) for name in Country_ENG]

# Add a column with English names of countries to df.
mapEng = dict(zip(country_original, Country_ENG))
df["Country_ENG"] = df["countrycode"]
df["Country_ENG"] = df["Country_ENG"].map(mapEng, na_action = "ignore")

In [ ]:
# Clean up the names in region_NUTS1.
df["region_NUTS1"] = [capital_letters(str(name)) for name in df["region_NUTS1"]]

In [ ]:
# Change names of original columns for clarity.
df = df.rename(columns={"Serial": "ID", 
                        "Q1_1": "Q1", 
                        "countrycode": "Country_Original",
                        "region_NUTS1": "Region",
                        "region_NUTS2": "Subregion",
                       })

In [ ]:
# Final list of columns that will be used. 
print(list(df.columns))

['ID', 'Country_Original', 'Region', 'Subregion', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q11', 'Q14_1', 'Q14_2', 'Q14_3', 'Q14_4', 'Q14_5', 'Q14_6', 'Q16', 'Q18_1', 'Q18_2', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q25', 'QG32_1', 'QG32_2', 'QGQ34_1', 'QG34_2', 'QG36_1', 'QG36_2', 'QG38_1', 'QG38_2', 'QG38_3', 'QG38_4', 'QG38_5', 'QG38_6', 'D1', 'D4a', 'D4a_96_other', 'D4b_1', 'D5', 'D6', 'D6a', 'D7', 'D8', 'D9', 'D10', 'D11', 'D3', 'D4_1', 'hAge', 'D2_1', 'EDU', 'lang_BE', 'lang_est', 'lang_finn', 'lang_LT', 'EUPresident_correct_1', 'EUPresident_correct_2', 'EUPresident_correct_3', "Q10_Austrian People's Party_AT", 'Q10_Austrian Social Democratic Party_AT', 'Q10_NEOS - The New Austria and Liberal Forum_AT', 'Q10_The Greens_AT', 'Q10_Austrian Freedom Party_AT', 'Q10_Alliance for the Future of Austria_AT', 'Q10_Workers Party of Belgium_BE', 'Q10_Christian Democratic and Flemish Party_BE', 'Q10_Socialist Party Different_BE', 'Q10_Open Flemish Liberals and Democrats_BE', 'Q10_N

In [ ]:
# Create a new DataFrame that excludes party information.
df_noparty = df[['ID', 'Country_Original', 'Region', 'Subregion', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q11', 'Q14_1', 'Q14_2', 'Q14_3', 'Q14_4', 'Q14_5', 'Q14_6', 'Q16', 'Q18_1', 'Q18_2', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q25', 'QG32_1', 'QG32_2', 'QGQ34_1', 'QG34_2', 'QG36_1', 'QG36_2', 'QG38_1', 'QG38_2', 'QG38_3', 'QG38_4', 'QG38_5', 'QG38_6', 'D1', 'D4a', 'D4a_96_other', 'D4b_1', 'D5', 'D6', 'D6a', 'D7', 'D8', 'D9', 'D10', 'D11', 'D3', 'D4_1', 'hAge', 'D2_1', 'EDU', 'lang_BE', 'lang_est', 'lang_finn', 'lang_LT', 'EUPresident_correct_1', 'EUPresident_correct_2', 'EUPresident_correct_3', 'Country_ENG']]

In [ ]:
# Export the cleaned data.
df.to_csv("cleandata_parties.csv", index=False)
df_noparty.to_csv("cleandata_noparty.csv", index=False)

These two .csv files can be now used for modelling. 

I have decided to create two different tables as it might be useful to have them separate for modelling purposes (due to the size of cleandata_parties.csv and the degree of missing values there).